http://localhost:8889/notebooks/git/google-quest-challenge/working/QstBertBase0113_1.ipynb

# start

In [1]:
trntmstmp=1584412344
valtmstmp=1585069785
import datetime
print([datetime.datetime.fromtimestamp(o).strftime('%Y-%m-%d %H:%M:%S') for o in (trntmstmp, valtmstmp)])

grand_total=1.5e8
MIN_TM_TRN=1580947200
MIN_TM_TST=1581552000
print([datetime.datetime.fromtimestamp(o).strftime('%Y-%m-%d %H:%M:%S') for o in (MIN_TM_TRN, MIN_TM_TST)])


CHNKSZ=1e6
POST_RATE_WANTED=0.1

['2020-03-17 02:32:24', '2020-03-24 17:09:45']
['2020-02-06 00:00:00', '2020-02-13 00:00:00']


# setup

In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)
import datetime
def dtnow(): return datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

SEED=101
HOME='/data/git/recsys20'
p_in=f'{HOME}/input'

import torch
from transformers import *
import torch
device=torch.device('cpu')
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader


pretrained_weights='bert-base-multilingual-cased'
bertmodel = BertModel.from_pretrained(pretrained_weights, output_hidden_states=True)
tokenizer = BertTokenizer.from_pretrained(pretrained_weights, do_lower_case=False)


In [3]:
cols=[
'toks',
'hshtgs',
'twtid',
'media',
'links',
'domns',
'twttyp',
'lang',
'tm',

'u1id',
'u1_fllwer_cnt',
'u1_fllwng_cnt',
'u1_vrfed',
'u1_create_tm',

'u2id',
'u2_fllwer_cnt',
'u2_fllwng_cnt',
'u2_vrfed',
'u2_create_tm',

'u1_fllw_u2',
'reply_tm',
'retwt_tm',
'retwt_cmmnt_tm',
'like_tm',
]
cols_cat = ['twttyp','lang']
cols_val = cols[:-4]
cols_tgt_tmstmp=[
    'retwt_tm',
    'reply_tm',
    'like_tm',
    'retwt_cmmnt_tm',
]
cols_tgt=[o.split('_tm')[0] for o in cols_tgt_tmstmp]
tgts             = ['Retweet','Reply','Like','RTwCmnt',]
assert cols_tgt == ['retwt',  'reply','like','retwt_cmmnt',]
ntgts=len(tgts)


tgt2col=dict(zip(tgts,cols_tgt))
tgt2col

{'Retweet': 'retwt',
 'Reply': 'reply',
 'Like': 'like',
 'RTwCmnt': 'retwt_cmmnt'}

# prep

In [4]:
%%time
dftrn=pd.read_csv(
    f'{p_in}/trn_{trntmstmp}.tsv',
    sep='\x01', header=None, 
    encoding='utf-8', 
    names=cols, 
    nrows=1e4
)

CPU times: user 75.1 ms, sys: 19.9 ms, total: 95 ms
Wall time: 94.1 ms


In [5]:
# tokenizer.pad_token, tokenizer.pad_token_id
# ('[PAD]', 0)

# tokenizer.sep_token, tokenizer.sep_token_id
# ('[SEP]', 102)
maxlen=512
def mkids(x):
    tokids=list(map(int, x.split('\t')))
    l=len(tokids) 
    if l<=maxlen: 
        return tokids + [0]*(maxlen-len(tokids))
    else: 
        return tokids[:maxlen-1]+[102]

In [6]:
def mk_tensors(df, istrn=True):
    tokids=dftrn.toks.apply(lambda x: mkids(x))
    Xarr=np.array(list(tokids))
    X=torch.tensor(Xarr,dtype=torch.long)
    if not istrn: return X
    ys=dftrn[cols_tgt_tmstmp].notna().values
    ys=torch.tensor(ys,dtype=torch.float)
    return X,ys
    

In [7]:
X,ys = mk_tensors(dftrn)

In [8]:
BS=8
ds = TensorDataset(X,ys)
dl = DataLoader(ds, batch_size=BS, shuffle=True)
for step, batch in enumerate(dl):
    X_b,ys_b = (o.to(device) for o in batch)
    print(X_b.shape,ys_b.shape)
    break

torch.Size([8, 512]) torch.Size([8, 4])


# model

In [9]:
EPOCHS=1
WD=0.01
LR=1e-5
SCHDLR_FUNC = get_cosine_schedule_with_warmup 
WARMUP_RATE = 0.05
N_CYCLS = .5
EPS = 1e-6


In [10]:
BS=16
ds = TensorDataset(X,ys)
dl = DataLoader(ds, batch_size=BS, shuffle=True)
for step, batch in enumerate(dl):
    X_b,ys_b = (o.to(device) for o in batch)
    print(X_b.shape,ys_b.shape)
    break

torch.Size([16, 512]) torch.Size([16, 4])


In [11]:
# https://huggingface.co/transformers/model_doc/bert.html#bertmodel
bertmodel=bertmodel.eval()
with torch.no_grad():
    last_hidden_state, pooler_output, hidden_states = bertmodel(X_b,)
    avg_pool = torch.mean(last_hidden_state,1)
    max_pool,_ = torch.max(last_hidden_state,1)
last_hidden_state.shape,pooler_output.shape, len(hidden_states), avg_pool.shape, max_pool.shape

(torch.Size([16, 512, 768]),
 torch.Size([16, 768]),
 13,
 torch.Size([16, 768]),
 torch.Size([16, 768]))

In [12]:
# see RobertaClassificationHead in transformers/modeling_roberta.py
N_HIDDEN = 768 
class TwtModel(nn.Module):
    def __init__(self, bertmodel, num_labels=4):
        super(TwtModel, self).__init__()
        self.bertmodel = bertmodel
        for param in self.bertmodel.parameters():
            param.requires_grad=False
        
        dense_size = 64
        self.relu=nn.ReLU(inplace=True)
        nx = N_HIDDEN*2
        self.dense = nn.Linear(nx, nx)
        self.dropout = nn.Dropout(0.1)
        self.out_proj = nn.Linear(nx, num_labels)
        
    def forward(self, x):
        attn_msk = (x!=tokenizer.pad_token_id).float()
        segments = torch.zeros(x.shape, dtype=torch.long)
        last_hidden_state, pooler_output, hidden_states = self.bertmodel(x, 
                                                                         attention_mask=attn_msk,
                                         token_type_ids=segments)
        avg_pool = torch.mean(last_hidden_state,1)
        max_pool,_ = torch.max(last_hidden_state,1)
        x = torch.cat([avg_pool,max_pool],1) 
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        return x

In [13]:
model=TwtModel(bertmodel)
model = model.to(device)

# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': WD},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]

optimizer = AdamW(optimizer_grouped_parameters, lr=LR, eps=EPS)
t_total = int(EPOCHS*len(dl))
scheduler = SCHDLR_FUNC(
    optimizer, 
    num_warmup_steps=int(WARMUP_RATE*t_total), 
    num_training_steps=t_total,
    num_cycles=N_CYCLS,
)


In [14]:
for epoch in range(EPOCHS):
    print(dtnow(), f'epoch {epoch} starts')
    y_tr_epoch = []
    y_pred_tr_epoch = []
    for step, batch in enumerate(dl):
        model.train()
        x_b,ys_b = (o.to(device) for o in batch)
        yb_pred = model(x_b)

        loss =  F.binary_cross_entropy_with_logits(yb_pred,ys_b)
        loss.backward()
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        model.zero_grad()
        print(loss.item())


2020-03-26 03:06:00 epoch 0 starts
0.724166750907898
0.7611744403839111
0.7165363430976868
0.7270771861076355
0.7215524315834045
0.7184446454048157
0.7295160889625549
0.703510582447052
0.7019727230072021
0.6832834482192993
0.6831183433532715
0.6756281852722168
0.6698161959648132
0.6478674411773682
0.664527416229248
0.6329403519630432
0.6270857453346252
0.5989986062049866
0.6107409596443176
0.6088334321975708
0.58661949634552
0.577872633934021
0.5494076609611511
0.5302914977073669
0.5388901829719543
0.5078228116035461
0.4997655153274536
0.5121878981590271
0.5350878238677979
0.4489264488220215
0.46939677000045776
0.4414101541042328
0.4342990219593048
0.4353935420513153
0.4303920865058899
0.38269665837287903
0.3928782641887665
0.34978458285331726
0.37044528126716614
0.38987016677856445
0.39276182651519775
0.34186574816703796
0.4189632833003998
0.3607929050922394
0.3236846327781677
0.29945361614227295
0.30355846881866455
0.33683884143829346
0.32145896553993225
0.3413238823413849
0.29785397

0.35026487708091736
0.3562433123588562
0.3695516884326935
0.23794475197792053
0.2594481110572815
0.2334095686674118
0.25155967473983765
0.32329070568084717
0.29399463534355164
0.3138405978679657
0.21448245644569397
0.25654351711273193
0.30383944511413574
0.2033945471048355
0.37577003240585327
0.2122085839509964
0.2366200089454651
0.29897525906562805
0.23352020978927612
0.25522392988204956
0.33408334851264954
0.3212050795555115
0.29248306155204773
0.4375634789466858
0.2571515142917633
0.3343764543533325
0.3281973898410797
0.24249199032783508
0.3124220669269562
0.3425847291946411
0.3493194878101349
0.3449494540691376
0.4042837619781494
0.2834582030773163
0.4252440333366394
0.38038337230682373
0.3264071047306061
0.24662239849567413
0.3277108073234558
0.33027446269989014
0.2710770070552826
0.355884313583374
0.29588741064071655
0.3233395218849182
0.24451042711734772
0.3657400608062744
0.2413473278284073
0.2256113737821579
0.30429700016975403
0.36991217732429504
0.2797745168209076
0.25420391